In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
import ast

In [39]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/final/users_final_data.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/final/products_final_data.csv')
original_products = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/data/products.csv')

## Building baseline 2 tower model

### The first issue is that for each training batch, we need to have the same amount of user-item pairs as input. This means we need to use some sort of sampling for each batch in order to make sure they're both the same size.

In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot, BatchNormalization
from tensorflow.keras.models import Model

In [18]:
user_input = Input(shape=(16,), name='user_input')
item_input = Input(shape=(30,), name='item_input')

In [19]:
#Changed from baseline
user_tower = Dense(128, activation='relu')(user_input)
user_tower = BatchNormalization()(user_tower)

In [20]:
item_tower = Dense(128, activation='relu')(item_input)
item_tower = BatchNormalization()(item_tower)

In [21]:
dot_product = Dot(axes=1)([user_tower, item_tower])

In [22]:
model = Model(inputs=[user_input, item_input], outputs=dot_product)

In [23]:
model.compile(optimizer='adam', loss='mse')

In [24]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 30)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 128)       │      2,176 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │      3,968 │ item_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_4[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128)       │        512 │ dense_5[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 1)         │          0 │ batch_normalizat… │
│                     │                   │            │ batch_normalizat… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 7,168 (28.00 KB)

 Trainable params: 6,656 (26.00 KB)

 Non-trainable params: 512 (2.00 KB)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def create_labels_and_train(users_df, products_df, model, batch_size, num_epochs):
    for epoch in range(num_epochs):
        # initilize the target similarity for the batch
        target_similarity = []

        # we're making the target similarity balanced, so there's an equal number of posivie and negetive indices in each batch
        num_indices = batch_size // 2

        # generating 1/2 batch size of random pairs, where there are positive indices (user and product have the same ID)
        positive_user_indices = np.random.randint(0, len(users_df), size=num_indices)
        # initialize storage of positive indicies
        positive_product_indices = []
        # loop over every user
        for user_idx in positive_user_indices:
            # locating product IDs in the user dataframe for the user we sampled
            user_product_id = users_df.iloc[user_idx]['product_id']
            # finding matching products in the products dataframe
            matching_products = products_df[products_df['product_id'] == user_product_id]
            # append the matching product to the positive product indices
            positive_product_indices.append(matching_products.index[0])

        # Generate random negative pairs (user and product have different product_ids)
        negative_user_indices = np.random.randint(0, len(users_df), size=num_indices)
        #print("NEGATIVE USER INDICES: ", negative_user_indices)
        negative_product_indices = []
        for user_idx in negative_user_indices:
            user_product_id = users_df.iloc[user_idx]['product_id']
            # find a product that doesn't have a matching product id
            non_matching_products = products_df[products_df['product_id'] != user_product_id]
            # append that to the negetive indicies
            negative_product_indices.append(non_matching_products.sample(1).index[0])

        # combining both positive and negetive indicies
        user_indices = np.concatenate([positive_user_indices, negative_user_indices])
        product_indices = np.concatenate([positive_product_indices, negative_product_indices])

        # create target similarity labels for the positive and negetive pairs
        target_similarity.extend([1] * num_indices)  # Positive pairs
        target_similarity.extend([0] * num_indices)  # Negative pairs
        target_similarity = np.array(target_similarity)

        # get the positive & negetive user data
        user_data = users_df.iloc[user_indices]
        user_ids = user_data['user_id'].tolist()
        product_data = products_df.iloc[product_indices]
        item_ids = product_data['product_id'].tolist()

        user_data = user_data.drop(columns=['product_id', 'user_id'])
        product_data = product_data.drop(columns=['product_id', 'flattened_name_embedding', 'flattened_brand_embedding'])

        # Train the model with the pairs
        model.fit([user_data, product_data], target_similarity, epochs=1, batch_size=batch_size, verbose=False)
        predicted_probabilities = model.predict([user_data, product_data]).flatten()
        user_item_predictions = list(zip(user_ids, item_ids, predicted_probabilities, target_similarity))
        #for user_id, item_id, predicted_prob, target_sim in user_item_predictions:
        #    print(f"User ID: {user_id}, Item ID: {item_id}, Predicted Probability: {predicted_prob:.4f}, Target Similarity: {target_sim}")

# Parameters
batch_size = 250
num_epochs = 20

create_labels_and_train(users_final, products_final, model, batch_size, num_epochs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 945us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 928us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 958us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 963us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 975us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 968us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 951us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 946us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step


The above code splits the data into training and testing data, and trains the model.

## Generating Recommendations

In [30]:
products_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13800 entries, 0 to 13799
Data columns (total 33 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   cost                                    13800 non-null  float64
 1   retail_price                            13800 non-null  float64
 2   product_id                              13800 non-null  int64  
 3   flattened_name_embedding                13800 non-null  object 
 4   flattened_brand_embedding               13800 non-null  object 
 5   department_Men                          13800 non-null  int64  
 6   department_Women                        13800 non-null  int64  
 7   category_Accessories                    13800 non-null  int64  
 8   category_Active                         13800 non-null  int64  
 9   category_Blazers & Jackets              13800 non-null  int64  
 10  category_Clothing Sets                  13800 non-null  in

In [61]:
def generate_recommendations(test_users_df, products_df, model, top_n=10):
    # Randomly select a user
    random_user_row = test_users_df.sample(1)
    random_user_id = random_user_row['user_id'].values[0]
    print(f"Generating recommendations for user ID: {random_user_id}...")

    # Prepare user data for the selected user
    user_data = random_user_row.drop(columns=['product_id', 'user_id']).values
    user_data_repeated = np.repeat(user_data, len(products_df), axis=0)

    # Prepare product data
    product_data = products_df.drop(columns=['product_id', 'flattened_name_embedding', 'flattened_brand_embedding']).values

    # Predict probabilities
    predicted_probabilities = model.predict([user_data_repeated, product_data]).flatten()

    # Sort product recommendations by increasing probability
    sorted_indices = np.argsort(predicted_probabilities)
    sorted_products = products_df.iloc[sorted_indices]

    # Display top N recommendations
    top_recommendations = sorted_products.head(top_n)
    #print("Top recommendations (sorted by increasing probability of interaction):")
    #print(top_recommendations[['product_id']])

    #Returns a list of the top n product IDs
    return top_recommendations[['product_id']]['product_id'].tolist()

In [62]:
original_products.columns

Index(['id', 'cost', 'category', 'name', 'brand', 'retail_price', 'department',
       'sku', 'distribution_center_id'],
      dtype='object')

In [66]:
def lookup_product_name(product_ids):
    for id in product_ids:
        product_row = original_products[original_products['id'] == id]
        name = product_row['name']
        print(f"ID = {id}, Name = {name}")

    

In [67]:
recs = generate_recommendations(users_final, products_final, model, top_n=10)
lookup_product_name(recs)

Generating recommendations for user ID: 86362...
 59/432 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step

432/432 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step
ID = 13629, Name = 28484    Solid Color Leather Adjustable Skinny Belt with
Name: name, dtype: object
ID = 14202, Name = 11644    GENUINE LEATHER SNAP ON STUDDED WHITE PIANO BE...
Name: name, dtype: object
ID = 14298, Name = 16355    Classic Tear Drop Mirror Lens Aviator Sunglasses
Name: name, dtype: object
ID = 12536, Name = 13235    Individual Bra Extenders
Name: name, dtype: object
ID = 13606, Name = 11091    Elegant PASHMINA SCARF WRAP SHAWL STOLE
Name: name, dtype: object
ID = 28700, Name = 23027    Wayfarer Style Sunglasses Dark Lens Black Frame
Name: name, dtype: object
ID = 9204, Name = 21579    Pink Ribbon Breast Cancer Awareness Knee High ...
Name: name, dtype: object
ID = 3049, Name = 21577    Pink Ribbon Breast Cancer Awareness Knee High ...
Name: name, dtype: object
ID = 14235, Name = 1401    Indestructable Aluminum Aluma Wallet - RED
Name: name, dtype: object
ID = 13607, Name = 11092    Cashmere Feel Unisex Pashmina Scarf - Plaid